In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet


class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return ap_distance, an_distance


class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.
    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.
    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs, **kwargs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]


class EmbeddingModel:
    def __init__(self, target_shape):
        base_cnn = resnet.ResNet50(
            weights="imagenet", input_shape=target_shape + (3,), include_top=False
        )

        flatten = layers.Flatten()(base_cnn.output)
        dense1 = layers.Dense(512, activation="relu")(flatten)
        dense1 = layers.BatchNormalization()(dense1)
        dense2 = layers.Dense(256, activation="relu")(dense1)
        dense2 = layers.BatchNormalization()(dense2)
        output = layers.Dense(256)(dense2)

        self.embedding = Model(base_cnn.input, output, name="Embedding")

        trainable = False
        for layer in base_cnn.layers:
            if layer.name == "conv5_block1_out":
                trainable = True
            layer.trainable = trainable

        anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
        positive_input = layers.Input(name="positive", shape=target_shape + (3,))
        negative_input = layers.Input(name="negative", shape=target_shape + (3,))

        distances = DistanceLayer()(
             self.embedding(resnet.preprocess_input(anchor_input)),
             self.embedding(resnet.preprocess_input(positive_input)),
             self.embedding(resnet.preprocess_input(negative_input)),
        ) 

        self.siamese_network = Model(
            inputs=[anchor_input, positive_input, negative_input], outputs=distances
        )

    def get_siamese_network(self):
        return self.siamese_network

    @staticmethod
    def train(siamese_network, train_data, val_data):
        siamese_model = SiameseModel(siamese_network)
        siamese_model.compile(optimizer=optimizers.Adam(0.0001))
        siamese_model.fit(train_data, epochs=10, validation_data=val_data)

    def test(self):
        """todo"""

    def inference(self):
        """todo"""

In [3]:
import pandas as pd
import tensorflow as tf
from random import choice, sample, randint
from tqdm import tqdm

FULL_DATASET_FILE = 'Ebay_info.txt'


class DatasetHandler:
    """DataHandler - класс для получения и обработки StandfordDataset для модели основанной на TripletLoss
    Attributes:
    -----------
    dataset_dir : str
            путь к папке Standford_Online_Products (Пример: ../Data/Standford_Online_Products)
    split_dataset : tuple(int, int)
            отношение частей train и test
    batch_size : int
            количество триплетов в батче
    target_shape : tuple(int, int)
            размер в который будут переведены изображения
    """

    def __init__(self, dataset_dir,
                 split_dataset=(0.8, 0.2),
                 dataset_part=1,
                 batch_size=64,
                 target_shape=(400, 400)):

        self.__target_shape = target_shape
        self.dataset_dir = dataset_dir

        full_dataset = pd.read_csv(f'{self.dataset_dir}/{FULL_DATASET_FILE}', sep=' ')
        self.__dataset_partitions = split_dataset

        self.df_train, self.df_test = self.__split_dataset(full_dataset, dataset_part)

    def get_train(self):
        return self.df_train
    def get_test(self):
        return self.df_test
        # Train/test triplets
    def Train_test_triplets(self,batch_size=64):
        tqdm.write(f'Train generating')
        train_triplets = self.__generate_triplets(self.df_train)
        self.train_dataset = self.__seal_dataset(train_triplets)
        self.train_dataset = self.train_dataset.batch(batch_size).prefetch(2)
        tqdm.write(f'Test generating')
        test_triplets = self.__generate_triplets(self.df_test)
        self.test_dataset = self.__seal_dataset(test_triplets)
        self.test_dataset = self.test_dataset.batch(batch_size).prefetch(2)
        return(self.train_dataset,self.test_dataset)

    def __split_dataset(self, data: pd.DataFrame, dataset_part: float):
        """
        Деление всего датасета на train/test в зависимости с dataset_part и split_dataset, переданным в параметры класса
        """
        super_classes = list(data.super_class_id.unique())
        df_train = data.copy()
        df_test = data.copy()
        for super_class in super_classes:
            super_class_indexes = list(data.loc[data.super_class_id == super_class].index)
            dropped_index = sample(super_class_indexes, int(dataset_part * len(super_class_indexes)))
            dropped_index = list(set(super_class_indexes) - set(dropped_index))
            train_super_class_indexes = sample(super_class_indexes,
                                               int(self.__dataset_partitions[0] * len(super_class_indexes)))

            test_super_class_indexes = list(set(super_class_indexes) - set(train_super_class_indexes))

            df_train.drop(index=test_super_class_indexes + dropped_index, inplace=True)
            df_test.drop(index=train_super_class_indexes + dropped_index, inplace=True)
        return df_train, df_test

    def __form_triplet(self, ind: int, data: pd.DataFrame):
        """
        Формирование триплета. Для выбранного изображения берется изображение из его класса, если такое отсутствует, то
        берется из суперласса. Отличное от выбранного изображение берется таким, чтобы оно не было в том же классе, что
        и выбранный.
        """
        anchor = data.iloc[ind]
        similar_indexes = data.loc[(data.class_id == anchor.class_id) & (data.image_id != anchor.image_id)].index
        if len(similar_indexes) == 0:
            similar_indexes = data.loc[(data.super_class_id == anchor.super_class_id)].index
        positive = data.loc[choice(similar_indexes)]
        different_indexes = data.drop(index=data.loc[data.class_id == anchor.class_id].index).index
        negative = data.loc[choice(different_indexes)]

        return anchor, positive, negative

    def __generate_triplets(self, data: pd.DataFrame):
        """
        Генерация триплетов, на данном этапе хранятся лишь пути к изображениям
        """
        triplets = {'anchors': [], 'positive': [], 'negative': []}
        for i in tqdm(range(data.shape[0])):
            anchor, positive, negative = self.__form_triplet(i, data)
            triplets['anchors'].append(f'{self.dataset_dir}{anchor["path"]}')
            triplets['positive'].append(f'{self.dataset_dir}{positive["path"]}')
            triplets['negative'].append(f'{self.dataset_dir}{negative["path"]}')
        return triplets

    def __seal_dataset(self, data: dict):
        """
        Получение триплета из целевого изображения, похожего на него и отличного от него.
        """
        anchor_dataset = tf.data.Dataset.from_tensor_slices(data['anchors'])
        positive_dataset = tf.data.Dataset.from_tensor_slices(data['positive'])
        negative_dataset = tf.data.Dataset.from_tensor_slices(data['negative'])

        triplets_path_dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
        triplets_images_dataset = triplets_path_dataset.map(self.__preprocess_triplets).map(
            self.__augmentation_triplets)

        return triplets_images_dataset

    def __augmentation_triplets(self, anchor, positive, negative):

        """
        Аугментация каждого изображения из триплета
        """

        return (
            self.__augmentation_image(anchor),
            self.__augmentation_image(positive),
            self.__augmentation_image(negative),
        )

    def __augmentation_image(self, image):

        """
        Аугментация изображения
        random_flip_left_right - случайное отражение по оси Y
        random_flip_up_down - случайное отражение по оси X
        random_brightness - случайное изменение яяркости
        random_contrast - случайное изменение контраста
        random_saturation -  случайное изменение насыщенности
        rot90 - переворот на 90 градусов случайное кол-во раз
        """

        aug_image = tf.image.random_flip_left_right(image)
        aug_image = tf.image.random_flip_up_down(aug_image)
        aug_image = tf.image.random_brightness(aug_image, max_delta=0.3)
        aug_image = tf.image.random_contrast(aug_image, lower=0.6, upper=1)
        aug_image = tf.image.random_saturation(aug_image, 0.6, 1)
        aug_image = tf.image.rot90(aug_image, k=randint(0, 3))
        return aug_image

    def preprocess_image(self, filename: tf.Tensor):
        """
        Загрузка изображения, декодирование, перевод значений в числа с плавающей точкой, а также изменение размера
        """

        image_string = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image_string, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, self.__target_shape)

        return image

    @tf.autograph.experimental.do_not_convert
    def __preprocess_triplets(self, anchor, positive, negative):
        """
        Метод для обработки каждого изображения из триплета
        """

        return (
            self.__preprocess_image(anchor),
            self.__preprocess_image(positive),
            self.__preprocess_image(negative),
        )

    """
    Метод для поучения установленного размера изображений
    """

    def get_target_shape(self):
        return self.__target_shape

    """
    Метод ждя получения train dataset
    """


In [4]:
Main_Path="C:/Users/user/Desktop/imp/stag2/"#только с полным путем у меня работает

In [ ]:
train_data,val_data=dataset_handler.Train_test_triplets()

In [5]:
embedding_model=tf.keras.models.load_model(Main_Path+"embedding_model")

In [6]:
dataset_dir=Main_Path+'Stanford_Online_Products'
dataset_handler=DatasetHandler(dataset_dir)
test=dataset_handler.get_test()
test_model =EmbeddingModel(target_shape=dataset_handler.get_target_shape())
test.reset_index(drop=False, inplace=True)

In [8]:
import os
import numpy as np
embeddings=pd.DataFrame([])
n=len(test)
#n=10
for i in range (n):#работает очень медленно, но я не знаю как оптимизировать
    #print('C:/Users/user/Desktop/imp/stag2/Stanford_Online_Products/'+test.iloc[i,-1])
    ret=tf.Variable(dataset_dir+'/'+test.iloc[i,-1],dtype=tf.string)
    #print(ret)
    #print(__preprocess_image(ret))
    im=dataset_handler.preprocess_image(ret)
    embeddings =embeddings.append(pd.DataFrame(test_model.embedding(tf.expand_dims(im, axis=0)).numpy()),ignore_index=True)
    #test_model. если бы была бы embedding_model, то было бы embedding_model.
df=embeddings.join(test.iloc[:n,-3:])
df.columns = df.columns.astype (str)
df.to_parquet('df_emb',engine='fastparquet')

In [9]:
df

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,class_id,super_class_id,path
0,-0.085142,2.263834,-0.885343,1.836391,-1.162681,-0.809887,0.831483,0.599229,-1.235901,-0.049218,...,2.126033,0.377122,1.116078,1.337602,0.849155,-1.622025,0.011362,1,1,bicycle_final/111085122871_1.JPG
1,0.288717,2.206600,-1.110308,1.920529,-0.854689,-0.343830,0.883807,0.728295,-0.981223,0.094597,...,2.286892,-0.117384,0.656719,1.288287,0.557438,-1.345682,0.175416,2,1,bicycle_final/111265328556_4.JPG
2,-0.196385,2.095917,-1.520651,1.525144,-1.129098,-0.837190,0.725680,0.675833,-1.076203,0.041220,...,2.077071,-0.269411,1.009172,1.579502,0.765996,-1.766425,-0.367796,2,1,bicycle_final/111265328556_6.JPG
3,0.202026,1.984765,-0.719857,1.333729,-1.253502,-0.486325,0.618138,0.398293,-1.160885,0.438762,...,2.302423,-0.056354,0.943287,1.618322,0.441846,-0.972010,0.076793,3,1,bicycle_final/111265348817_2.JPG
4,0.018245,2.131875,-0.822429,1.775731,-1.151801,-0.687859,0.592101,0.729311,-1.227695,0.014845,...,2.170035,-0.032085,0.965099,1.746426,0.802978,-1.460227,-0.450101,4,1,bicycle_final/111375616144_1.JPG
5,0.161554,1.965031,-1.204765,1.795040,-1.166557,-0.501812,0.895801,0.637940,-1.066792,0.187091,...,2.450968,0.061530,0.956310,1.411432,0.832268,-1.613709,0.332984,6,1,bicycle_final/111476691561_5.JPG
6,0.416950,2.344188,-0.906669,1.560650,-1.334940,-0.673229,0.586341,0.766725,-0.541149,-0.213924,...,1.598669,0.171077,0.241735,1.178248,0.673523,-1.590650,0.360401,6,1,bicycle_final/111476691561_9.JPG
7,-0.233066,1.984723,-1.523679,1.706785,-1.199485,-0.460579,1.060521,0.725634,-1.059003,-0.133839,...,2.178601,-0.127449,1.177631,1.272139,0.571926,-1.800723,-0.039683,7,1,bicycle_final/111502876121_4.JPG
8,-0.087598,1.791137,-0.584348,1.810316,-1.511694,0.214525,0.818769,0.736485,-1.107280,0.149841,...,2.042173,-0.197772,1.251872,0.979754,0.455892,-1.094271,0.194153,7,1,bicycle_final/111502876121_8.JPG
9,0.123475,2.057287,-0.632529,1.940392,-1.289135,-0.534737,1.011053,0.923829,-1.224715,-0.239091,...,1.954331,0.413944,0.997784,1.598377,0.662749,-1.817938,-0.122325,9,1,bicycle_final/111579336659_3.JPG


In [10]:
!pip install fastparquet
df.to_parquet('df_emb',engine='fastparquet')